# To recap

### Data so far

1. We have created a list of people and their ORCID identifiers - [output/pure_person_to_orcid.txt](output/pure_person_to_orcid.txt)
2. We have created a list of ORCID to PubMed identifiers - [output/orcid.tsv](output/orcid.tsv)
3. We have created a list of PubMed IDs to PubMed info - [output/pubmed.tsv](output/pubmed.tsv)
4. We have created a list of the top 100 terms for each person - [output/orcid-tf-idf.txt](output/orcid-tf-idf.txt)

### Questions

Now we need to use the common identifiers in these data to answer some questions:

1. Can we produce a set of potential collaborators for each person, a collaborator being someone they have significant terms in common with but not previously published with.
2. Can we select a set of people that most closely map to a specific piece of text.


### Pandas dataframes

Normally at this point we would use a database. However, to keep it all in a single language and framework, and as the data themselves are relatively small, we can use pandas - https://pandas.pydata.org/

>pandas is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.



So, following on from the data descriptions above:


In [ ]:
import pandas as pd

# 1. People and their ORCID identifiers 
personToOrcid = pd.read_csv('data/pure_person_to_orcid.txt',sep='\t')
print(personToOrcid.shape)
print(personToOrcid.head())

# 2. ORCID to PubMed identifiers
orcidToPubmed = pd.read_csv('data/orcid.tsv',sep='\t')
print(orcidToPubmed.shape)
print(orcidToPubmed.head())

# 3. PubMed IDs to PubMed info
pubmedToInfo = pd.read_csv('data/pubmed.tsv',sep='\t')
print(pubmedToInfo.shape)
print(pubmedToInfo.head())

# 4. Top 100 terms for each person
topTerms = pd.read_csv('data/orcid-tf-idf.txt',sep='\t')
print(topTerms.shape)
print(topTerms.head())


A first pass would be to count the number of matching terms between each person. To start with, let's just compare the first in the list to everyone else:

In [ ]:
 #loop through people (limit to top 1)
def compare_people(limit):
    for i, iData in personToOrcid.head(n=limit).iterrows():
        print('### Comparing',i,iData.pure_person_id,iData.orcid_id)
        #get all terms and tf-idf values
        iTopTerms=topTerms[topTerms['orcid_id']==iData.orcid_id]
        #print(iTopTerms.head())
        iTerms = iTopTerms['term']

        for j in range(i+1,personToOrcid.shape[0]):
            jData=personToOrcid.iloc[j]
            jTopTerms=topTerms[topTerms['orcid_id']==jData.orcid_id]
            #print(jTopTerms['term'].head())
            jTerms = jTopTerms['term']
            #print(iTerms,jTerms)
            com = list(set(iTerms) & set(jTerms))
            #only show maches with more than 1 matching term
            if len(com) > 1:
                print(jData.orcid_id,com)
    
compare_people(1)        

There are few matches with many overlapping terms, except one **0000-0001-7086-8882**. Let's look at the research pages of these two people

https://research-information.bristol.ac.uk/en/persons/melody-a-s-sylvestre(81e7f06c-77d8-4020-9608-0b30dd001c43)/publications.html https://research-information.bristol.ac.uk/en/persons/nicholas-a-teanby(4ec18a96-fd2e-4311-a6fa-7ec65696a4e9)/publications.html 

This suggests they are from similar research areas, and have indeed co-published.   

Let's try again, this time with the top 3

In [ ]:
compare_people(3)  

The second person matched no-one, the third however matched quite a few. Let's modify the function to sort by number:  

In [ ]:
def compare_people(limit):
    for i, iData in personToOrcid.head(n=limit).iterrows():
        print('### Comparing',i,iData.pure_person_id,iData.orcid_id)
        #get all terms and tf-idf values
        iTopTerms=topTerms[topTerms['orcid_id']==iData.orcid_id]
        iTerms = iTopTerms['term']
        jComp={}
        for j in range(i+1,personToOrcid.shape[0]):
            jData=personToOrcid.iloc[j]
            jTopTerms=topTerms[topTerms['orcid_id']==jData.orcid_id]
            jTerms = jTopTerms['term']
            com = list(set(iTerms) & set(jTerms))
            #only show maches with more than 1 matching term
            if len(com) > 1:
                jComp[jData.orcid_id]=com
        #create sorted dictionary using number of items
        jComp = sorted(jComp.items(), key=lambda kv: len(kv[1]), reverse=True)
        for p in jComp[0:10]:
            print(p)
            
compare_people(3)

This only shows us terms that are in common. It could be possible to have lots of terms in common, but these terms are not the 'most' representative of either person. It would be better to compare all terms, and their tf-idf values.

Back in the TF-IDF section (http://localhost:8888/lab#TF-IDF-using-sklearn) we created a similarity matrix from the TF-IDF model.



In [ ]:
#get similarity matrix for all people

matrixCom = {}

#get tfidf matrix and orcidText dictionary
%store -r matrix
%store -r token_dict

for i in range(0,len(matrix)):
    iOrcid=list(token_dict)[i]
    matrixCom[iOrcid]={}
    for j in range(0,len(matrix)):
        jOrcid=list(token_dict)[j]
        matrixCom[iOrcid][jOrcid]=matrix[i][j]

o = open('output/orcid-to-orcid-tf-idf.tsv','w')
o.write('orcid_1\torcid_2\ttf-idf\n')
counter=0
for m in matrixCom:
    sorted_res = sorted(matrixCom[m].items(), key=lambda kv: kv[1], reverse=True)
    if counter<3:
        print(m)
        for s in sorted_res[0:5]:
            print('\t',s[0],s[1])
        counter+=1
    for s in sorted_res:
        o.write(m+'\t'+s[0]+'\t'+str(s[1])+'\n')
o.close()


### Collaboration recommendation engine

This similarity data depicts the similarity between each person's publication text based on tf-idf. Often, similarities arise due to co-publication, and perhaps a more informative recommender would be to idenfity cases where people have signficant overlap in their publication text, but have never previously co-published. 

In [ ]:
#load the new data into a dataframe

orcidToOrcid = pd.read_csv('data/orcid-to-orcid-tf-idf.tsv',sep='\t')
print(orcidToOrcid.shape)
print(orcidToOrcid.head())

### Matching text to people

Using the same tf-idf model, we can try and match any piece of text to the most relevant people.

In [ ]:
#https://stackoverflow.com/questions/55677314/using-sklearn-how-do-i-calculate-the-tf-idf-cosine-similarity-between-documents
%store -r tfs
corpus_tfidf = tfs
%store -r tfidf

In [35]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#26930047
query="""
Diagnosis of Coronary Heart Diseases Using Gene Expression Profiling; Stable Coronary Artery Disease, Cardiac Ischemia with and without Myocardial Necrosis.    
Cardiovascular disease (including coronary artery disease and myocardial infarction) is one of the leading causes of death in Europe, and is influenced by 
both environmental and genetic factors. With the recent advances in genomic tools and technologies there is potential to predict and diagnose heart 
disease using molecular data from analysis of blood cells. We analyzed gene expression data from blood samples taken from normal people (n = 21), 
non-significant coronary artery disease (n = 93), patients with unstable angina (n = 16), stable coronary artery disease (n = 14) and myocardial 
infarction (MI; n = 207). We used a feature selection approach to identify a set of gene expression variables which successfully differentiate different 
cardiovascular diseases. The initial features were discovered by fitting a linear model for each probe set across all arrays of normal individuals and 
patients with myocardial infarction. Three different feature optimisation algorithms were devised which identified two discriminating sets of genes, 
one using MI and normal controls (total genes = 6) and another one using MI and unstable angina patients (total genes = 7). In all our classification 
approaches we used a non-parametric k-nearest neighbour (KNN) classification method (k = 3). The results proved the diagnostic robustness of the final 
feature sets in discriminating patients with myocardial infarction from healthy controls. Interestingly it also showed efficacy in discriminating 
myocardial infarction patients from patients with clinical symptoms of cardiac ischemia but no myocardial necrosis or stable coronary artery disease, 
despite the influence of batch effects and different microarray gene chips and platforms."
"""
query_tfidf = tfidf.transform([query])
cosineSimilarities = cosine_similarity(query_tfidf, corpus_tfidf).flatten()

print(len(cosineSimilarities))
cosineData={}
for c in range(0,len(cosineSimilarities)):
    cosineData[c]=cosineSimilarities[c]
sortedCosineData = sorted(cosineData.items(), key=lambda kv: kv[1], reverse=True)
for s in sortedCosineData[0:10]:
    print(s,list(token_dict)[s[0]])


529
(353, 0.1498055959132002) 0000-0002-2515-0852
(0, 0.14495921424895872) 0000-0003-0924-3247
(321, 0.14106425254783597) 0000-0002-1753-3730
(506, 0.09991253199428538) 0000-0003-4638-601X
(396, 0.08854748685568083) 0000-0001-7240-4563
(140, 0.0878923678564203) 0000-0001-6080-480X
(16, 0.08521416949481442) 0000-0002-8193-9559
(494, 0.07221759249818203) 0000-0002-3180-6993
(50, 0.06675288084209205) 0000-0003-2278-5892
(231, 0.06434504244860838) 0000-0001-6648-3007
